# **Import thư viện**

In [1]:
import math
import tkinter as tk
from tkinter import messagebox

# **Khởi tạo hằng số**

In [2]:
# Kích thước bàn cờ
BOARD_SIZE = 9

# Số quân liên tiếp cần để thắng
WIN_CONDITION = 4

# Ký hiệu ô trống và hai người chơi
EMPTY = '.'
PLAYER = 'X'
COMPUTER = 'O'

# Độ sâu tối đa khi tìm kiếm Alpha-Beta
DEPTH_LIMIT = 3

# Các hướng di chuyển (8 hướng: dọc, ngang, chéo)
DIRECTIONS = [(-1, -1), (-1, 0), (-1, 1), (0, -1),
              (0, 1), (1, -1), (1, 0), (1, 1)]

CELL_SIZE = 40

PADDING = 20

WINDOW_SIZE = BOARD_SIZE * CELL_SIZE + PADDING * 2

# **CLASS BOARD**

In [3]:
class Board:
    def __init__(self):
        # Khởi tạo bàn cờ với tất cả ô là trống
        self.board = [[EMPTY for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

    def display(self):
        # In ra bàn cờ hiện tại kèm chỉ số hàng và cột
        print("  " + " ".join(str(i) for i in range(BOARD_SIZE)))
        for idx, row in enumerate(self.board):
            print(f"{idx} " + " ".join(row))

    def is_valid_move(self, row, col):
        # Kiểm tra ô (row, col) có hợp lệ (trong bàn cờ và đang trống) hay không
        return 0 <= row < BOARD_SIZE and 0 <= col < BOARD_SIZE and self.board[row][col] == EMPTY

    def make_move(self, row, col, player):
        # Đặt quân của player vào vị trí (row, col)
        self.board[row][col] = player

    def undo_move(self, row, col):
        # Hoàn tác (xóa) quân cờ tại vị trí (row, col) (đặt lại thành trống)
        self.board[row][col] = EMPTY

    def is_full(self):
        # Kiểm tra bàn cờ đã đầy chưa (không còn ô trống)
        return all(cell != EMPTY for row in self.board for cell in row)

    def check_win(self, player):
        # Kiểm tra xem player có thắng không bằng cách dò các chuỗi liên tiếp đủ WIN_CONDITION
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                for dr, dc in DIRECTIONS:
                    # Nếu tìm thấy chuỗi đủ dài thì player thắng
                    if self.count_in_direction(r, c, dr, dc, player) >= WIN_CONDITION:
                        return True
        return False

    def count_in_direction(self, r, c, dr, dc, player):
        # Đếm số quân liên tiếp của player bắt đầu từ (r,c) theo hướng (dr,dc)
        count = 0
        for i in range(WIN_CONDITION):
            nr, nc = r + dr * i, c + dc * i
            # Nếu vị trí trong bàn và là quân của player thì tăng đếm
            if 0 <= nr < BOARD_SIZE and 0 <= nc < BOARD_SIZE and self.board[nr][nc] == player:
                count += 1
            else:
                break
        return count

    def get_valid_moves(self):
        # Tìm các nước đi trống trong phạm vi 1 ô xung quanh các quân đã đánh
        moves = set()
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                if self.board[r][c] != EMPTY:
                    for dr in [-1, 0, 1]:
                        for dc in [-1, 0, 1]:
                            if dr == 0 and dc == 0:
                                continue
                            nr, nc = r + dr, c + dc
                            if 0 <= nr < BOARD_SIZE and 0 <= nc < BOARD_SIZE and self.board[nr][nc] == EMPTY:
                                moves.add((nr, nc))
        if not moves:
            return [(BOARD_SIZE // 2, BOARD_SIZE // 2)]
        return list(moves)

# **CLASS AI**

In [4]:
class AI:
    def __init__(self, board):
        # Lưu tham chiếu tới bàn cờ hiện tại
        self.board = board

    def evaluate_line(self, line, player):
        # Đánh giá giá trị một đoạn 4 ô (line) với góc nhìn của player
        opp = PLAYER if player == COMPUTER else COMPUTER  # Đối thủ
        line_str = ''.join(line)
        score = 0

        # Nếu player có 4 quân liên tiếp => điểm cao nhất
        if player * 4 in line_str:
            return 100000
        elif player * 3 in line_str:
            score += 1000
        elif player * 2 in line_str:
            score += 100
        elif player in line_str:
            score += 10

        # Nếu đối thủ có 4 quân liên tiếp => điểm âm lớn (nguy hiểm)
        if opp * 4 in line_str:
            return -100000
        elif opp * 3 in line_str:
            score -= 800
        elif opp * 2 in line_str:
            score -= 80
        elif opp in line_str:
            score -= 8

        return score


    def evaluate_board(self, player):
        # Đánh giá tổng thể bàn cờ với góc nhìn của player
        total = 0
        b = self.board.board

        # Đánh giá theo hàng ngang
        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE - 3):
                row = [b[r][c + i] for i in range(4)]
                total += self.evaluate_line(row, player)

        # Đánh giá theo cột dọc
        for c in range(BOARD_SIZE):
            for r in range(BOARD_SIZE - 3):
                col = [b[r + i][c] for i in range(4)]
                total += self.evaluate_line(col, player)

        # Đánh giá theo 2 đường chéo (chéo chính và chéo phụ)
        for r in range(BOARD_SIZE - 3):
            for c in range(BOARD_SIZE - 3):
                diag1 = [b[r + i][c + i] for i in range(4)]        # chéo chính
                diag2 = [b[r + 3 - i][c + i] for i in range(4)]    # chéo phụ
                total += self.evaluate_line(diag1, player)
                total += self.evaluate_line(diag2, player)
        return total

    def sort_moves(self, moves, player):
        # Sắp xếp nước đi dựa trên giá trị đánh giá tạm thời để tăng hiệu quả cắt tỉa alpha-beta
        def score(move):
            r, c = move
            self.board.make_move(r, c, player)
            val = self.evaluate_board(player)
            self.board.undo_move(r, c)             # Hoàn tác
            return -val
        return sorted(moves, key=score)

    def alpha_beta(self, depth, alpha, beta, maximizing_player):
        # Thuật toán tìm kiếm Alpha-Beta cắt tỉa
        if self.board.check_win(COMPUTER):
            return 100000
        if self.board.check_win(PLAYER):
            return -100000
        if depth == 0 or self.board.is_full():
            # Đạt độ sâu tối đa hoặc bàn cờ đầy, đánh giá trạng thái hiện tại
            return self.evaluate_board(COMPUTER)
        # Lấy danh sách nước đi hợp lệ đã được sắp xếp
        moves = self.sort_moves(self.board.get_valid_moves(), COMPUTER if maximizing_player else PLAYER)

        if maximizing_player:
            max_eval = -math.inf
            for r, c in moves:
                self.board.make_move(r, c, COMPUTER)
                eval = self.alpha_beta(depth - 1, alpha, beta, False)
                self.board.undo_move(r, c)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:   # Cắt tỉa nếu không cần thiết tiếp tục
                    break
            return max_eval
        else:
            min_eval = math.inf
            for r, c in moves:
                self.board.make_move(r, c, PLAYER)
                eval = self.alpha_beta(depth - 1, alpha, beta, True)
                self.board.undo_move(r, c)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:   # Cắt tỉa
                    break
            return min_eval

    def find_immediate_threats(self, player):
        threats = []
        opp = PLAYER if player == COMPUTER else COMPUTER

        directions = DIRECTIONS
        b = self.board.board

        for r in range(BOARD_SIZE):
            for c in range(BOARD_SIZE):
                if b[r][c] != EMPTY:
                    continue
                for dr, dc in directions:
                    # Kiểm tra theo hướng (dr, dc) nếu đặt vào (r,c)
                    b[r][c] = player
                    line = []
                    for i in range(-4, 5):
                        nr, nc = r + dr * i, c + dc * i
                        if 0 <= nr < BOARD_SIZE and 0 <= nc < BOARD_SIZE:
                            line.append(b[nr][nc])
                        else:
                            line.append('#')  # Ký hiệu biên
                    b[r][c] = EMPTY  # Hoàn tác

                    line_str = ''.join(line)
                    if player * 4 in line_str or player * 3 in line_str:
                        threats.append((r, c))
                        break
        return threats


    def best_move(self):
        # Ưu tiên chặn nước đi nguy hiểm của đối thủ (nếu có thể thắng ở bước tiếp theo)
        opponent_moves = self.board.get_valid_moves()
        for r, c in opponent_moves:
            self.board.make_move(r, c, PLAYER)
            if self.board.check_win(PLAYER):
                self.board.undo_move(r, c)
                return (r, c)  # Chặn ngay lập tức
            self.board.undo_move(r, c)

        # Ưu tiên chặn các mối đe dọa (vd: ..XX hoặc .XXX.)
        threats = self.find_immediate_threats(PLAYER)
        if threats:
            return threats[0]


        # Tìm nước đi tốt nhất bằng Alpha-Beta
        best_val = -math.inf
        best = None
        moves = self.sort_moves(self.board.get_valid_moves(), COMPUTER)
        for r, c in moves:
            self.board.make_move(r, c, COMPUTER)
            move_val = self.alpha_beta(DEPTH_LIMIT - 1, -math.inf, math.inf, False)
            self.board.undo_move(r, c)
            if move_val > best_val:
                best_val = move_val
                best = (r, c)
        return best

# **CLASS GAME**

In [5]:
class Game:
    def __init__(self):
        # Khởi tạo trò chơi với bàn cờ mới và AI, người chơi đi trước
        self.board = Board()
        self.ai = AI(self.board)
        self.turn = PLAYER  # Nếu muốn máy đi trước thì gán COMPUTER

    def play(self):
        # Vòng lặp chính của trò chơi
        while True:
            self.board.display()

            # Kiểm tra người chơi thắng
            if self.board.check_win(PLAYER):
                print("You win!")
                break
            # Kiểm tra máy thắng
            if self.board.check_win(COMPUTER):
                print("Computer wins!")
                break
            # Kiểm tra hòa
            if self.board.is_full():
                print("Draw!")
                break

            if self.turn == PLAYER:
                # Lượt người chơi nhập nước đi
                try:
                    move = input("Enter your move (row col): ")
                    row, col = map(int, move.strip().split())
                    if not self.board.is_valid_move(row, col):
                        print("Invalid move. Try again.")
                        continue
                    self.board.make_move(row, col, PLAYER)
                    self.turn = COMPUTER
                except Exception:
                    # Nếu nhập sai định dạng hoặc giá trị
                    print("Invalid input. Try again.")
            else:
                # Lượt máy tính
                print("Computer is thinking...")
                r, c = self.ai.best_move()
                self.board.make_move(r, c, COMPUTER)
                self.turn = PLAYER

# **CLASS GAMEGUI**

In [6]:
class GameGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("TIC TAC TOE 9x9")
        self.canvas = tk.Canvas(root, width=WINDOW_SIZE, height=WINDOW_SIZE)
        self.canvas.pack()
        self.game = Game()
        self.canvas.bind("<Button-1>", self.on_click)
        self.draw_board()

    def draw_board(self):
        self.canvas.delete("all")
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                x0 = PADDING + j * CELL_SIZE
                y0 = PADDING + i * CELL_SIZE
                x1 = x0 + CELL_SIZE
                y1 = y0 + CELL_SIZE
                self.canvas.create_rectangle(x0, y0, x1, y1, outline="black")
                mark = self.game.board.board[i][j]
                if mark != EMPTY:
                    self.canvas.create_text((x0 + x1) // 2, (y0 + y1) // 2,
                                            text=mark, font=("Arial", 18))

    def on_click(self, event):
        if self.game.turn != PLAYER:
            return  # Chờ lượt máy

        col = (event.x - PADDING) // CELL_SIZE
        row = (event.y - PADDING) // CELL_SIZE

        if not self.game.board.is_valid_move(row, col):
            return

        self.game.board.make_move(row, col, PLAYER)
        self.game.turn = COMPUTER
        self.draw_board()
        self.check_game_over()

        # Để máy đánh sau một chút
        self.root.after(500, self.ai_move)

    def ai_move(self):
        if self.game.turn == COMPUTER:
            move = self.game.ai.best_move()
            if move:
                r, c = move
                self.game.board.make_move(r, c, COMPUTER)
            self.game.turn = PLAYER
            self.draw_board()
            self.check_game_over()

    def check_game_over(self):
        if self.game.board.check_win(PLAYER):
            messagebox.showinfo("END", "You win!")
            self.root.quit()
        elif self.game.board.check_win(COMPUTER):
            messagebox.showinfo("END", "Computer win!")
            self.root.quit()
        elif self.game.board.is_full():
            messagebox.showinfo("END", "DRAW!")
            self.root.quit()

# **CLASS GAMEMANAGER**

In [7]:
class GameManager:
    def __init__(self):
        pass

    def play_console(self):
        # Khởi chạy game trên console.
        game = Game()
        game.play()

    def play_gui(self):
        # Khởi chạy game với giao diện đồ họa.
        root = tk.Tk()
        gui = GameGUI(root)
        root.mainloop()

    def main_menu(self):
        # Hiển thị menu lựa chọn chế độ chơi.
        while True:
            print("\n MENU")
            print("1. Nhập tọa độ chơi trên console")
            print("2. Chơi trên giao diện GUI")
            print("3. Thoát")
            choice = input("Chọn (1-3):" \
            "\n1. Nhập tọa độ chơi trên console" \
            "\n2. Chơi trên giao diện GUI" \
            "\n3. Thoát").strip()
            if choice == '1':
                self.play_console()
            elif choice == '2':
                self.play_gui()
            elif choice == '3':
                print("Thoát chương trình.")
                break
            else:
                print("Lựa chọn không hợp lệ. Vui lòng chọn lại.")

# **MAIN**

In [ ]:
if __name__ == "__main__":
    manager = GameManager()
    manager.main_menu()


 MENU
1. Nhập tọa độ chơi trên console
2. Chơi trên giao diện GUI
3. Thoát
  0 1 2 3 4 5 6 7 8
0 . . . . . . . . .
1 . . . . . . . . .
2 . . . . . . . . .
3 . . . . . . . . .
4 . . . . . . . . .
5 . . . . . . . . .
6 . . . . . . . . .
7 . . . . . . . . .
8 . . . . . . . . .
  0 1 2 3 4 5 6 7 8
0 . . . . . . . . .
1 . . . . . . . . .
2 . . . . . . . . .
3 . . . . X . . . .
4 . . . . . . . . .
5 . . . . . . . . .
6 . . . . . . . . .
7 . . . . . . . . .
8 . . . . . . . . .
Computer is thinking...
  0 1 2 3 4 5 6 7 8
0 . . . . . . . . .
1 . . . . . . . . .
2 . . . . . . . . .
3 . . . . X . . . .
4 . . . . O . . . .
5 . . . . . . . . .
6 . . . . . . . . .
7 . . . . . . . . .
8 . . . . . . . . .
Invalid move. Try again.
  0 1 2 3 4 5 6 7 8
0 . . . . . . . . .
1 . . . . . . . . .
2 . . . . . . . . .
3 . . . . X . . . .
4 . . . . O . . . .
5 . . . . . . . . .
6 . . . . . . . . .
7 . . . . . . . . .
8 . . . . . . . . .
Invalid input. Try again.
  0 1 2 3 4 5 6 7 8
0 . . . . . . . . .
1 . . . .